In [2]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [3]:
from collections import defaultdict

In [4]:
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i+1] = p

In [5]:
price[12]

0

## Getting the max splitting by dynamic programming

In [6]:
def r(n):
    return(max([price[n]] + [r(i) + r(n-i) for i in range(1,n)]))

In [7]:
r(15)

45

## Optimize

In [8]:
from functools import wraps

In [9]:
def memo(f):
    memo.already_computed = {}
    @wraps(f) # decorator after python 2.7 and python 3.6
    def _wrap(arg):
        result = None
        
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        
        return result
    return _wrap

In [10]:
solution = {}

In [11]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue
    """
    max_price, max_split = max([(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1,n)]\
                              , key = lambda x: x[0])
    
    solution[n] = (n - max_split, max_split)
    
    return max_price

In [12]:
r(20)

60

In [13]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3)}

## Parse solution

In [14]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0:
        return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [15]:
r(234)

743

In [16]:
' '.join(\
    map(str,parse_solution(234)))

'11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 3'

## Edit distance

In [17]:
solution = {}

In [18]:
from functools import lru_cache

In [20]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    if len(string1)==0: return len(string2)
    if len(string2)==0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
#     print(tail_s2)
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2))
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB: {} = {}'\
                        .format(tail_s1, tail_s2))
        
    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key = lambda x: x[0])
    
    solution[(string1, string2)] = operation
#     print(operation)
    return min_distance

In [21]:
edit_distance('ABCDE', 'ABCCEF')

2

In [22]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB: D = C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F'}

## HomewParse solution

In [23]:
def parse_edit_distance_solution(string1, string2):
    if len(string1) > 0:
        temp = solution[(string1, string2)]
        parsed_solution.append(temp)

        if temp.startswith('ADD'):
            parse_edit_distance_solution(string1, string2[:-1])
        elif temp.startswith('DEL'):
            parse_edit_distance_solution(string1[:-1], string2)
#         elif temp.startswith('SUB'):
#             parse_edit_distance_solution(string1[:-1]+temp.split(' ')[1], string2)
        else:
            parse_edit_distance_solution(string1[:-1], string2[:-1])

        return [i for i in parsed_solution if i != '']

In [24]:
parsed_solution = []
string1 = 'ABCDE'
string2 = 'ABCCEF'
clear_solution = parse_edit_distance_solution(string1, string2)
print('-->'.join([string1] + clear_solution + [string2]))

ABCDE-->ADD F-->SUB: D = C-->ABCCEF


## Problem Case 3: Pinyin Auto Correction Problem

In [25]:
chinese_dataset = '../../article_9k.txt'
CHINESE_CHARACTERS = open(chinese_dataset, 'r').read()
print(CHINESE_CHARACTERS[:10])

此外自本周6月12日


In [26]:
# !pip install pinyin
import pinyin

In [27]:
pinyin.get('你好', format='strip', delimiter=' ')

'ni hao'

In [28]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [29]:
CHINESE_PINYIN_COPYS = chinese_to_pinyin(CHINESE_CHARACTERS)

In [30]:
len(CHINESE_PINYIN_COPYS)

129433034

In [31]:
import re

In [32]:
def tokens(text):
    return re.findall('[a-z]+', text.lower())

In [34]:
print(tokens(CHINESE_PINYIN_COPYS[:100]))

['ci', 'wai', 'zi', 'ben', 'zhou', 'yue', 'ri', 'qi', 'chu', 'xiao', 'mi', 'shou', 'ji', 'deng', 'kuan', 'ji', 'xing', 'wai', 'qi', 'yu', 'ji', 'xing', 'yi']


In [35]:
from collections import Counter, defaultdict

In [36]:
PINYIN_COUNT = Counter(tokens(CHINESE_PINYIN_COPYS))

In [37]:
def correct(word):
    candidates = (known(edits0(word)) or known(edits1(word))\
    or known(edits2(word)) or [word])
    
    return max(candidates, key = PINYIN_COUNT.get)
    

In [38]:
print(str(PINYIN_COUNT))

Counter({'shi': 860634, 'de': 809887, 'n': 691114, 'yi': 682478, 'ji': 645276, 'guo': 430042, 'zhong': 409418, 'zhi': 398612, 'xin': 359619, 'li': 355441, 'zai': 334106, 'wei': 326301, 'hua': 304941, 'yu': 302949, 'she': 293312, 'he': 285689, 'bu': 281533, 'ri': 278379, 'jin': 278141, 'you': 277726, 'xian': 269047, 'gong': 259593, 'yue': 258519, 'ren': 257321, 'qi': 251164, 'yuan': 248823, 'jian': 248173, 'da': 247785, 'xing': 241741, 'jia': 239795, 'fa': 233137, 'nian': 231398, 'di': 221266, 'jing': 220462, 'xi': 217848, 'sheng': 211580, 'cheng': 210265, 'jie': 208675, 'er': 208539, 'zhe': 205224, 'ye': 197406, 'xiang': 196711, 'fu': 194850, 'wu': 194660, 'chang': 193611, 'zi': 192620, 'ge': 191779, 'zhu': 191577, 'hui': 187634, 'dui': 182885, 'shang': 178941, 'ti': 176853, 'shou': 175635, 'si': 175066, 'ke': 173705, 'ju': 173114, 'dai': 171074, 'dao': 167050, 'tong': 166443, 'le': 166313, 'guan': 164991, 'fang': 161433, 'zheng': 160688, 'sai': 158317, 'chu': 156005, 'zhan': 155206, '

In [39]:
def known(words):
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

### edits1

In [40]:
def edits1(word):
    pairs = splits(word)
    deletes = [a + b[1:] for (a, b) in pairs if b]
    transposes = [a + b[1] + b[0] + b[2:] for (a, b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a, b) in pairs for c in alphabets if b]
    inserts = [a+c+b for (a, b) in pairs for c in alphabets]
    
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) for i in range(len(word) + 1)]

alphabets = 'abcdefghijklmnopqrstuvwxyz'
    

In [41]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [43]:
[i for i in edits1('yin') if i in PINYIN_COUNT]

['qin',
 'nin',
 'yun',
 'yi',
 'ying',
 'lin',
 'yin',
 'bin',
 'xin',
 'pin',
 'yan',
 'jin',
 'min']

In [45]:
correct('pign')

'ping'

In [46]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [47]:
correct_sequence_pinyin('zhe shi yi ge ce shi')

'zhe shi yi ge ce shi'

In [48]:
correct_sequence_pinyin('wo xiang shang qinng hau da xue')

'wo xiang shang qing hua da xue'

## 思考题-homework？ 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？

In [49]:
total_words_freq = sum(PINYIN_COUNT.values())

In [50]:
list(PINYIN_COUNT.items())[:10]

[('ci', 95927),
 ('wai', 131353),
 ('zi', 192620),
 ('ben', 55466),
 ('zhou', 80326),
 ('yue', 258519),
 ('ri', 278379),
 ('qi', 251164),
 ('chu', 156005),
 ('xiao', 126736)]

In [51]:
def get_word_prob(word):
    if word in PINYIN_COUNT:
        return PINYIN_COUNT[word] / total_words_freq
    else:
        return PINYIN_COUNT.most_common()[-1][-1] / total_words_freq

In [63]:
@lru_cache(maxsize=2**12)
def split_text_pinyin(pinyin_no_delim):
    n = len(pinyin_no_delim)
    max_prob, max_split = max([(get_word_prob(pinyin_no_delim), 0)] + \
                              [((split_text_pinyin(pinyin_no_delim[:i]) * \
                                split_text_pinyin(pinyin_no_delim[i:]))**0.5, i) for i in range(1,n)], key=lambda x: x[0])
#     max_prob, max_split = max([(get_word_prob(pinyin_no_delim), 0)] + \
#                               [((split_text_pinyin(correct(pinyin_no_delim[:i])) * \
#                                 split_text_pinyin(correct(pinyin_no_delim[i:])))**0.5, i) for i in range(1,n)], key=lambda x: x[0])
    split_text_solution[pinyin_no_delim] = (pinyin_no_delim[:max_split], pinyin_no_delim[max_split:])
    return max_prob

In [60]:
# get_word_prob('zheshiyigeceshi')
# (get_word_prob('zhe') * get_word_prob('shiyigeceshi'))**0.5
split_text_solution = defaultdict(list)
# sys.setrecursionlimit(150000)
split_text_pinyin('zheshiyigeceshi')

0.016525920631495016

In [57]:
import sys
def split_text_pinyin_parse(pinyin_no_delim):
    try:
        left_split, right_split = split_text_solution[pinyin_no_delim]

        if left_split == '': return [right_split]

        return split_text_pinyin_parse(left_split) + split_text_pinyin_parse(right_split)
    except ValueError:
        print(pinyin_no_delim)
        sys.exit(0)

In [61]:
# dict(split_text_solution)
split_text_pinyin_parse('zheshiyigeceshi')

['zhe', 'shi', 'yi', 'ge', 'ce', 'shi']

In [67]:
import gc
import functools

gc.collect()
wrappers = [
    a for a in gc.get_objects() 
    if isinstance(a, functools._lru_cache_wrapper)]

for wrapper in wrappers:
    wrapper.cache_clear()

In [68]:
split_text_solution = defaultdict(list)
test_string = 'woxiangshangqinnghuadaxue'
split_text_pinyin(test_string)
split_text_pinyin_parse(test_string)

['wo', 'xiang', 'shang', 'qi', 'n', 'n', 'g', 'hua', 'da', 'xue']

In [70]:
def split_text_pinyin_summary(pinyin_no_delim):
    @lru_cache(maxsize=2**10)
    def split_text_pinyin(pinyin_no_delim):
        n = len(pinyin_no_delim)
        max_prob, max_split = max([(get_word_prob(pinyin_no_delim), 0)] + \
                                  [((split_text_pinyin(pinyin_no_delim[:i]) * \
                                    split_text_pinyin(pinyin_no_delim[i:]))**0.5, i) for i in range(1,n)], key=lambda x: x[0])
        split_text_solution[pinyin_no_delim] = (pinyin_no_delim[:max_split], pinyin_no_delim[max_split:])
    #     print(dict(split_text_solution))
        return max_prob
    
    split_text_solution = defaultdict(list)
    test_string = pinyin_no_delim #'woxiangshangqinnghaudaxue'
    split_text_pinyin(test_string)
    return split_text_pinyin_parse(test_string)

split_text_pinyin_summary('woxiangshangqinnghuadaxue')

['wo', 'xiang', 'shang', 'qi', 'n', 'n', 'g', 'hua', 'da', 'xue']

In [71]:
def correct_sequence_pinyin_no_delim(text_pinyin):
    return ' '.join(map(correct, split_text_pinyin_summary(text_pinyin)))

In [72]:
correct_sequence_pinyin_no_delim('woxiangshangqinnghuadaxue')

'wo xiang shang qi n n g hua da xue'

## Summary

In the part of adjust text pinyin without delimiter, I try to use the one gram to separate them, and find that it performs worse in words such as 'qinng'. Maybe it would be better when using 2-gram model.